In [452]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import *
from pandas import DataFrame
import datetime


In [453]:
## Read in match dataset
df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')
df['year'] = pd.DatetimeIndex(df['match_date']).year

## Filter to only games from 2010 onwards. Remove shortened season stats (2019 preds and 2020 stats)

df = df.query('year >= 2012').query('year != 2020')\
        
## Read in Player Database
players = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\player_ages.csv')

## Merge game stats with player info
df['player_name'] = df['player_last_name'].astype(str) + ", " + df['player_first_name']
joined_data = df.merge(players, on=['player_name'],how='left')

## Calculate age on match day (take year of game less year of birth, so will remain same age all season)
joined_data['age'] = pd.DatetimeIndex(joined_data['match_date']).year - pd.DatetimeIndex(joined_data['dob']).year

# Write to csv to check
# joined_data.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl_limited.csv')

# Aggregate 
avg_scores = joined_data.groupby(['player_team', 'player_first_name', 'player_last_name','player_name', 'age', 'year'])\
.agg({'afl_fantasy_score': 'mean'})\
.sort_values(by=['afl_fantasy_score'], ascending=False)\

pd.set_option('display.max_rows', 10)
# print(avg_scores)
avg_scores.query('age < 32').head(10)



C:\Users\anoble\AppData\Local\Temp\ipykernel_9164\293218346.py:2: DtypeWarning: Columns (25,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')


afl_fantasy_score
player_team      player_first_name player_last_name player_name          age  year                   
Brisbane Lions   Tom               Rockliff         Rockliff, Tom        24.0 2014         134.777778
Collingwood      Dane              Swan             Swan, Dane           28.0 2012         130.666667
Hawthorn         Tom               Mitchell         Mitchell, Tom        25.0 2018         128.166667
                                                                         24.0 2017         127.181818
Collingwood      Brodie            Grundy           Grundy, Brodie       25.0 2019         124.166667
Western Bulldogs Jack              Macrae           Macrae, Jack         24.0 2018         122.631579
Gold Coast       Touk              Miller           Miller, Touk         25.0 2021         122.142857
St Kilda         Jack              Steele           Steele, Jack         26.0 2021         121.363636
Geelong          Patrick           Dangerfield      Dangerfield, Patrick 27.0 2017         121.083333
Adelaide         Rory              Laird            Laird, Rory          29.0 2022         120.350000

# Data cleaning 


In [454]:
## Remove COVID impacted years. Apply TOG > 60%. Filter to only scores in decent weather conditions

joined_data= joined_data\
        .query('year != 2020')\
        .query('time_on_ground_percentage > 60')\
        # .query('match_weather_type == ["MOSTLY_SUNNY", "OVERCAST","MOSTLY_CLEAR","SUNNY"]')

## Identify outlier scores (top / bottom 2 after applying first round of filters
joined_data['score_rank_asc'] = joined_data.groupby(['player_name', 'year'])['afl_fantasy_score'].rank(method="first", ascending=True)
joined_data['score_rank_desc'] = joined_data.groupby(['player_name', 'year'])['afl_fantasy_score'].rank(method="first", ascending=False)


matches_played = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_played=('year', 'mean'))

matches_played_min5 = matches_played.query('matches >=5')

joined_data = joined_data.merge(matches_played, on =['player_name','year'], how='inner')

# Remove outliers and seasons with fewer than 5 games

joined_data['Percentile'] = joined_data['score_rank_asc'] / joined_data['matches']

## Turn on if we want to remove outliers - runs risk of having limited data remaining to have >5 games sample
# joined_data = joined_data.query('0.05 <= Percentile <= 0.95')

joined_data

,Unnamed: 0.1,Unnamed: 0,venue_name,match_id,match_home_team,match_away_team,match_date,match_local_time,match_attendance,match_round,...,player_name,First Name,Last Name,dob,age,score_rank_asc,score_rank_desc,matches,year_played,Percentile
0,2939,600055,Gabba,14945,Brisbane Lions,Port Adelaide,30/07/2016,19:25:00,13085,19,...,"Smith, Archie",Archie,Smith,19/07/1995,21.0,3.0,1.0,3,2016.0,1.000000
1,2940,600450,Adelaide Oval,14954,Adelaide,Brisbane Lions,6/08/2016,19:10:00,43549,20,...,"Smith, Archie",Archie,Smith,19/07/1995,21.0,2.0,2.0,3,2016.0,0.666667
2,2941,600670,Gabba,14959,Brisbane Lions,Carlton,13/08/2016,13:45:00,17432,21,...,"Smith, Archie",Archie,Smith,19/07/1995,21.0,1.0,3.0,3,2016.0,0.333333
3,2942,605993,MCG,15080,Collingwood,Brisbane Lions,28/05/2017,13:10:00,32750,10,...,"Smith, Archie",Archie,Smith,19/07/1995,22.0,6.0,1.0,6,2017.0,1.000000
4,2943,606519,Gabba,15092,Brisbane Lions,Fremantle,10/06/2017,16:35:00,11742,12,...,"Smith, Archie",Archie,Smith,19/07/1995,22.0,4.0,2.0,6,2017.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96871,667146,664771,Optus Stadium,16567,Fremantle,Port Adelaide,20/08/2023,14:40:00,0,23,...,"Emmett, Tom",Tom,Emmett,30/11/2001,22.0,1.0,2.0,2,2023.0,0.500000
96872,667147,664863,MCG,16569,Hawthorn,Fremantle,26/08/2023,13:45:00,0,24,...,"Emmett, Tom",Tom,Emmett,30/11/2001,22.0,2.0,1.0,2,2023.0,1.000000
96873,667149,665001,GMHBA Stadium,16572,Geelong,Western Bulldogs,26/08/2023,19:25:00,0,24,...,"Conway, Toby",Toby,Conway,24/04/2003,20.0,1.0,1.0,1,2023.0,1.000000
96874,667150,665092,Adelaide Oval,16574,Port Adelaide,Richmond,27/08/2023,12:00:00,0,24,...,"Trezise, James",James,Trezise,15/06/2002,21.0,1.0,1.0,1,2023.0,1.000000


# Seasons active factor
 

In [455]:
seasons_active = joined_data.groupby(['player_name']).agg(years_active=('year_played', 'nunique'))

seasons_active

,years_active
player_name,
"Aarts, Jake",2
"Abbott, Ryan",2
"Ablett, Gary",8
"Acres, Blake",9
"Adams, Leigh",3
...,...
"de Boer, Matt",10
"van Berlo, Nathan",4
"van Rooyen, Jacob",1


## In Season change factors

In [456]:
# ## Add scores during 1st half of season. Convert match round to numeric, return AFL score if valid, else return Null
joined_data['h1_fantasy_score'] = np.where(pd.to_numeric(joined_data['match_round'], errors='coerce') <= 10, joined_data['afl_fantasy_score'], np.NaN)
joined_data['h2_fantasy_score'] = np.where(pd.to_numeric(joined_data['match_round'], errors='coerce') > 10, joined_data['afl_fantasy_score'], np.NaN)


# Test
joined_data.agg({'afl_fantasy_score': 'mean','h1_fantasy_score': 'mean','h2_fantasy_score': 'mean'})

afl_fantasy_score    72.335821
h1_fantasy_score     72.428320
h2_fantasy_score     72.393678
dtype: float64

## In game stat factors

In [457]:
matches_played_min5.merge(joined_data.drop(['matches','year_played'], axis = 1), on =['player_name'], how='inner')

,player_name,matches,year_played,Unnamed: 0.1,Unnamed: 0,venue_name,match_id,match_home_team,match_away_team,match_date,...,year,First Name,Last Name,dob,age,score_rank_asc,score_rank_desc,Percentile,h1_fantasy_score,h2_fantasy_score
0,"Aarts, Jake",19,2021.0,661119,636665,MCG,15875,Richmond,Carlton,18/03/2021,...,2021,Jake,Aarts,8/12/1994,27.0,9.0,11.0,0.473684,48.0,NaN
1,"Aarts, Jake",19,2021.0,661120,637352,MCG,15890,Hawthorn,Richmond,28/03/2021,...,2021,Jake,Aarts,8/12/1994,27.0,1.0,19.0,0.052632,19.0,NaN
2,"Aarts, Jake",19,2021.0,661121,637626,MCG,15896,Richmond,Sydney,3/04/2021,...,2021,Jake,Aarts,8/12/1994,27.0,7.0,13.0,0.368421,44.0,NaN
3,"Aarts, Jake",19,2021.0,661122,637950,Adelaide Oval,15903,Port Adelaide,Richmond,9/04/2021,...,2021,Jake,Aarts,8/12/1994,27.0,2.0,18.0,0.105263,22.0,NaN
4,"Aarts, Jake",19,2021.0,661123,638318,Marvel Stadium,15911,St Kilda,Richmond,15/04/2021,...,2021,Jake,Aarts,8/12/1994,27.0,12.0,8.0,0.631579,55.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633829,"vandenBerg, Aaron",7,2018.0,631674,619927,MCG,15397,Melbourne,Greater Western Sydney,26/08/2018,...,2018,NaN,NaN,NaN,NaN,7.0,1.0,1.000000,NaN,103.0
633830,"vandenBerg, Aaron",7,2018.0,631675,620058,MCG,15400,Melbourne,Geelong,7/09/2018,...,2018,NaN,NaN,NaN,NaN,6.0,2.0,0.857143,NaN,NaN
633831,"vandenBerg, Aaron",7,2018.0,631676,620190,MCG,15403,Hawthorn,Melbourne,14/09/2018,...,2018,NaN,NaN,NaN,NaN,2.0,6.0,0.285714,NaN,NaN
633832,"vandenBerg, Aaron",7,2018.0,631677,620324,Optus Stadium,15406,West Coast,Melbourne,22/09/2018,...,2018,NaN,NaN,NaN,NaN,5.0,3.0,0.714286,NaN,NaN


In [458]:
#Explode player data by years so that each year matches to every other active year
player_summ_exp = matches_played_min5.merge(joined_data.drop(['matches','year_played'], axis = 1), on =['player_name'], how='inner')\
                

## Limit to games in the same year for 1yr factors, as well as 3 years up to the predicted season, and 5 years up to predicted season
player_summ_exp_1yr = player_summ_exp.query(('year_played == year'))
player_summ_exp_3yr = player_summ_exp.query(('year_played < year + 3')).query(('year_played >= year'))
player_summ_exp_5yr = player_summ_exp.query(('year_played < year + 5')).query(('year_played >= year'))

print(player_summ_exp.shape)
print(player_summ_exp_1yr.shape)
print(player_summ_exp_3yr.shape)
print(player_summ_exp_5yr.shape)

## Test matching works 
pd.set_option('display.max_rows', 50)
player_summ_exp_3yr.groupby(['year_played','year']).agg(countall=('player_name', 'count')).head(50)

(633834, 95)
(93754, 95)
(212092, 95)
(288199, 95)


countall
year_played year          
2012.0      2012      8191
2013.0      2012      7155
            2013      8192
2014.0      2012      6363
            2013      7002
            2014      8199
2015.0      2013      6327
            2014      7202
            2015      8215
2016.0      2014      6208
            2015      6885
            2016      8704
2017.0      2015      6430
            2016      7508
            2017      8731
2018.0      2016      6540
            2017      7440
            2018      8701
2019.0      2017      6904
            2018      7572
            2019      8758
2021.0      2019      6990
            2021      8661
2022.0      2021      7434
            2022      8573
2023.0      2021      6857
            2022      7521
            2023      8829

In [459]:
## Average factors over 1 year of game data
model_factors_agg_1yr = player_summ_exp_1yr.groupby(['player_name', 'year_played'])\
.agg({'player_height_cm': 'mean',
'player_weight_kg': 'mean',
'age': 'mean',
'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_1yr.columns = [str(col) + '_1yr' for col in model_factors_agg_1yr.columns]


In [460]:
# Build 3 Year Player Metrics
model_factors_agg_3yr = player_summ_exp_3yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_3yr.columns = [str(col) + '_3yr' for col in model_factors_agg_3yr.columns]

model_factors_agg_3yr

kicks_3yr  marks_3yr  handballs_3yr  \
player_name       year_played                                        
Aarts, Jake       2021.0        6.263158   2.736842       5.210526   
Ablett, Gary      2012.0       19.450000   3.050000      14.300000   
                  2013.0       17.853659   3.268293      14.585366   
                  2014.0       17.654545   2.963636      14.927273   
                  2015.0       16.800000   2.800000      14.450000   
...                                  ...        ...            ...   
van Berlo, Nathan 2015.0       10.184211   3.815789       6.026316   
van Rooyen, Jacob 2023.0        4.578947   3.000000       4.052632   
vandenBerg, Aaron 2015.0       11.000000   3.416667       7.083333   
                  2016.0        8.153846   3.230769       8.076923   
                  2018.0        6.714286   3.476190       8.333333   

                               disposals_3yr  effective_disposals_3yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           11.473684                 8.684211   
Ablett, Gary      2012.0           33.750000                23.789474   
                  2013.0           32.439024                22.275000   
                  2014.0           32.581818                22.351852   
                  2015.0           31.250000                21.475000   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           16.210526                10.421053   
van Rooyen, Jacob 2023.0            8.631579                 6.684211   
vandenBerg, Aaron 2015.0           18.083333                10.666667   
                  2016.0           16.230769                10.192308   
                  2018.0           15.047619                10.142857   

                               disposal_efficiency_percentage_3yr  goals_3yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                73.210526   0.947368   
Ablett, Gary      2012.0                                69.000000   1.300000   
                  2013.0                                68.097561   1.317073   
                  2014.0                                68.381818   1.418182   
                  2015.0                                69.350000   1.550000   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                64.578947   0.315789   
van Rooyen, Jacob 2023.0                                78.210526   1.421053   
vandenBerg, Aaron 2015.0                                57.500000   0.416667   
                  2016.0                                62.500000   0.653846   
                  2018.0                                67.714286   0.809524   

                               behinds_3yr  hitouts_3yr  tackles_3yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2021.0          0.578947     0.000000     2.684211  ...   
Ablett, Gary      2012.0          0.950000     0.000000     5.500000  ...   
                  2013.0          1.024390     0.000000     5.024390  ...   
                  2014.0          0.890909     0.000000     5.327273  ...   
                  2015.0          0.825000     0.000000     5.250000  ...   
...                                    ...          ...          ...  ...   
van Berlo, Nathan 2015.0          0.394737     0.078947     3.578947  ...   
van Rooyen, Jacob 2023.0          0.421053     3.631579     2.210526  ...   
vandenBerg, Aaron 2015.0          0.916667     0.083333     5.333333  ...   
                  2016.0          0.769231     0.076923     4.384615  ...   
                  2018.0          0.714286     0.047619     3.428571  ...   

                               marks_on_lead_3yr  pressure_acts_3yr  \
player_name       year_played                           

In [461]:
# Build 5 Year Player Metrics

model_factors_agg_5yr = player_summ_exp_5yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_5yr.columns = [str(col) + '_5yr' for col in model_factors_agg_5yr.columns]

model_factors_agg_5yr

kicks_5yr  marks_5yr  handballs_5yr  \
player_name       year_played                                        
Aarts, Jake       2021.0        6.263158   2.736842       5.210526   
Ablett, Gary      2012.0       19.450000   3.050000      14.300000   
                  2013.0       17.853659   3.268293      14.585366   
                  2014.0       17.654545   2.963636      14.927273   
                  2015.0       17.683333   2.883333      14.400000   
...                                  ...        ...            ...   
van Berlo, Nathan 2015.0       11.000000   4.098361       6.327869   
van Rooyen, Jacob 2023.0        4.578947   3.000000       4.052632   
vandenBerg, Aaron 2015.0       11.000000   3.416667       7.083333   
                  2016.0        8.153846   3.230769       8.076923   
                  2018.0        8.272727   3.454545       7.878788   

                               disposals_5yr  effective_disposals_5yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           11.473684                 8.684211   
Ablett, Gary      2012.0           33.750000                23.789474   
                  2013.0           32.439024                22.275000   
                  2014.0           32.581818                22.351852   
                  2015.0           32.083333                22.220339   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           17.327869                11.283333   
van Rooyen, Jacob 2023.0            8.631579                 6.684211   
vandenBerg, Aaron 2015.0           18.083333                10.666667   
                  2016.0           16.230769                10.192308   
                  2018.0           16.151515                10.333333   

                               disposal_efficiency_percentage_5yr  goals_5yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                73.210526   0.947368   
Ablett, Gary      2012.0                                69.000000   1.300000   
                  2013.0                                68.097561   1.317073   
                  2014.0                                68.381818   1.418182   
                  2015.0                                69.233333   1.466667   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                66.491803   0.409836   
van Rooyen, Jacob 2023.0                                78.210526   1.421053   
vandenBerg, Aaron 2015.0                                57.500000   0.416667   
                  2016.0                                62.500000   0.653846   
                  2018.0                                64.000000   0.666667   

                               behinds_5yr  hitouts_5yr  tackles_5yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2021.0          0.578947     0.000000     2.684211  ...   
Ablett, Gary      2012.0          0.950000     0.000000     5.500000  ...   
                  2013.0          1.024390     0.000000     5.024390  ...   
                  2014.0          0.890909     0.000000     5.327273  ...   
                  2015.0          0.866667     0.000000     5.333333  ...   
...                                    ...          ...          ...  ...   
van Berlo, Nathan 2015.0          0.409836     0.065574     4.245902  ...   
van Rooyen, Jacob 2023.0          0.421053     3.631579     2.210526  ...   
vandenBerg, Aaron 2015.0          0.916667     0.083333     5.333333  ...   
                  2016.0          0.769231     0.076923     4.384615  ...   
                  2018.0          0.787879     0.060606     4.121212  ...   

                               marks_on_lead_5yr  pressure_acts_5yr  \
player_name       year_played                           

In [462]:
# Most common position for the year
# Calculate most common position played over past year
# Consider grouping if needed to remove specificity (e.g. Left and Right indicators probably not useful)


# Games player per year. Filter to only where games >= 5
# position_mapp = joined_data.groupby(['player_name','year']).agg({'match_date': 'count'})\
#     .query('match_date >=5')

position_data = joined_data

position_data['position_grouped'] = np.select(
    [
     position_data['player_position'].isin(["FB"]), 
     position_data['player_position'].isin(["BPL", "BPR"]),
     position_data['player_position'].isin(["CHB"]), 
     position_data['player_position'].isin(["HBFL", "HBFR"]),
     position_data['player_position'].isin(["RK"]), 
     position_data['player_position'].isin(["C", "RR" ,"R"]),
     position_data['player_position'].isin(["WL", "WR"]), 
     position_data['player_position'].isin(["FF"]),
     position_data['player_position'].isin(["FPL", "FPR"]),
     position_data['player_position'].isin(["HFFL", "HFFR"]),
     position_data['player_position'].isin(["CHF"]),
     position_data['player_position'].isin(["INT", "SUB"])
    ],
    [
      'FB',
      'BP',
      'CHB',
      'HBF',
      'RUCK',
      'C',
      'W',
      'FF',
      'FP',
      'HFF',
      'CHF',
      'Ignore'  
    ], 
    default='Other'
     )

## Test no positions have not been allocated. Result = 0 rows
# joined_data.query('position_grouped in ["Other"]')\
#     .groupby(['player_position']).agg(matches=('match_date', 'count'))

position_data['matches'] = position_data.groupby(['player_name', 'year','position_grouped'])['match_id'].transform('count')

# Find most common position, ignoring sub and interchange scores
position_data['rank'] = position_data.query('position_grouped not in ["Ignore"]').groupby(['player_name', 'year'])['matches'].rank(method="first", ascending=False)

player_pos_yearly = position_data.query('rank == 1')[['player_name', 'year','position_grouped']]

player_pos_yearly['position_grouped_simple'] = np.select(
    [
     player_pos_yearly['position_grouped'].isin(["FB", "BP","FF","FP"]), 
     player_pos_yearly['position_grouped'].isin(["CHB","CHF","C"]), 
     player_pos_yearly['position_grouped'].isin(["HBF", "W", "HFF"]),
     player_pos_yearly['position_grouped'].isin(["RUCK"])
    ],
    [
      'Key_Position',
      'Centres',
      'Wings_Flank',
      'Ruck'
    ], 
    default='Other'
     )

# Test output
player_pos_yearly.query('player_name == "Himmelberg, Harry"')


,player_name,year,position_grouped,position_grouped_simple
77621,"Himmelberg, Harry",2016,FP,Key_Position
77626,"Himmelberg, Harry",2017,FP,Key_Position
77636,"Himmelberg, Harry",2018,FF,Key_Position
77661,"Himmelberg, Harry",2019,FF,Key_Position
77697,"Himmelberg, Harry",2021,CHF,Centres
77709,"Himmelberg, Harry",2022,CHF,Centres
77742,"Himmelberg, Harry",2023,CHB,Centres


## Response variable

In [463]:
# Average AFL fantasy score per season - remove 2019, Covid impacted
response_var = joined_data.query('year != 2020')\
    .groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_int=('year', 'mean'),avg_score=('afl_fantasy_score', 'mean'))\
    .query('matches >= 5').query('avg_score > 0')\
    .sort_values(by=['avg_score'], ascending=False)

# Subtract 1 year from AFL score so it matches to factors from last year
response_var['predicted_year'] = response_var['year_int'] - 1

response_var.query('player_name == "Yeo, Elliot"')

matches  year_int   avg_score  predicted_year
player_name year                                               
Yeo, Elliot 2018       25    2018.0  106.240000          2017.0
            2019       23    2019.0  101.434783          2018.0
            2017       23    2017.0   95.826087          2016.0
            2023        6    2023.0   84.333333          2022.0
            2021       12    2021.0   83.000000          2020.0
            2016       20    2016.0   78.950000          2015.0
            2015       24    2015.0   75.750000          2014.0
            2014       11    2014.0   70.181818          2013.0
            2013       19    2013.0   58.052632          2012.0
            2012        7    2012.0   52.857143          2011.0

## Join all factors together


In [464]:
## Player / year combos to include
factors = matches_played_min5.merge(seasons_active, left_on=['player_name'], right_on=['player_name'], how='inner')\
           .merge(player_pos_yearly, left_on=['player_name', 'year_played'], right_on=['player_name', 'year'], how='inner')\
           .merge(model_factors_agg_1yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_3yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_5yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')
            
factors.query('player_name == "Yeo, Elliot"')

,player_name,matches,year_played,years_active,year,position_grouped,position_grouped_simple,player_height_cm_1yr,player_weight_kg_1yr,age_1yr,...,marks_on_lead_5yr,pressure_acts_5yr,rating_points_5yr,ruck_contests_5yr,score_launches_5yr,shots_at_goal_5yr,spoils_5yr,afl_fantasy_score_5yr,h1_fantasy_score_5yr,h2_fantasy_score_5yr
5663,"Yeo, Elliot",7,2012.0,11,2012,HBF,Wings_Flank,191.0,91.0,19.0,...,0.142857,9.428571,6.314286,0.000000,0.857143,0.142857,1.857143,52.857143,54.000000,52.666667
5664,"Yeo, Elliot",19,2013.0,11,2013,HBF,Wings_Flank,191.0,91.0,20.0,...,0.076923,9.576923,7.515385,0.000000,0.846154,0.153846,2.038462,56.653846,54.700000,57.875000
5665,"Yeo, Elliot",11,2014.0,11,2014,HBF,Wings_Flank,191.0,91.0,21.0,...,0.083333,10.833333,8.472973,0.027778,0.861111,0.250000,1.916667,60.675676,59.894737,61.500000
5666,"Yeo, Elliot",24,2015.0,11,2015,W,Wings_Flank,191.0,91.0,22.0,...,0.135593,11.220339,9.585246,0.169492,1.016949,0.783333,1.830508,66.606557,69.178571,65.366667
5667,"Yeo, Elliot",20,2016.0,11,2016,W,Wings_Flank,191.0,91.0,23.0,...,0.126582,12.594937,9.201235,0.151899,1.101266,1.100000,1.607595,69.654321,72.027778,68.463415
5668,"Yeo, Elliot",23,2017.0,11,2017,HBF,Wings_Flank,191.0,91.0,24.0,...,0.147368,13.073684,10.574227,0.126316,1.115789,1.093750,1.589474,77.072165,80.377778,75.608696
5669,"Yeo, Elliot",25,2018.0,11,2018,C,Centres,191.0,91.0,25.0,...,0.227723,16.019802,11.882524,0.128713,1.356436,1.352941,1.306931,87.660194,89.456522,88.041667
5670,"Yeo, Elliot",23,2019.0,11,2019,C,Centres,191.0,91.0,26.0,...,0.236842,18.043860,12.695652,0.105263,1.508772,1.426087,1.175439,92.086957,96.893617,90.736842
5671,"Yeo, Elliot",12,2021.0,11,2021,C,Centres,191.0,91.0,28.0,...,0.253012,20.481928,14.240964,0.012048,1.626506,1.168675,0.939759,98.662651,103.300000,97.130435
5672,"Yeo, Elliot",6,2023.0,11,2023,C,Centres,191.0,91.0,30.0,...,0.159091,21.454545,13.665909,0.000000,1.704545,1.045455,0.909091,92.363636,99.545455,90.096774


In [465]:
## Player / year combos to include
factors = matches_played_min5.merge(seasons_active, left_on=['player_name'], right_on=['player_name'], how='inner')\
           .merge(player_pos_yearly, left_on=['player_name', 'year_played'], right_on=['player_name', 'year'], how='inner')\
           .merge(model_factors_agg_1yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_3yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_5yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(response_var, left_on=['player_name', 'year_played'], right_on=['player_name', 'predicted_year'], how='left')\


## Add delta factors. Use lazy difference between 1 year and 3 year average

factors['fantasy_score_yearly_change'] = (factors['afl_fantasy_score_1yr'] - factors['afl_fantasy_score_3yr']).fillna(0)
factors['fantasy_score_inyear_change'] = (factors['h2_fantasy_score_1yr'] - factors['h1_fantasy_score_1yr']).fillna(0)

factors = factors.drop(['h1_fantasy_score_1yr','h2_fantasy_score_1yr','h1_fantasy_score_3yr','h2_fantasy_score_3yr','h1_fantasy_score_5yr','h2_fantasy_score_5yr'], axis = 1)




## Join on response variable 
 

In [466]:
response_var

matches  year_int   avg_score  predicted_year
player_name     year                                               
Rockliff, Tom   2014       18    2014.0  134.777778          2013.0
Swan, Dane      2012       21    2012.0  130.666667          2011.0
Rockliff, Tom   2016       15    2016.0  128.400000          2015.0
Mitchell, Tom   2018       24    2018.0  128.166667          2017.0
                2017       22    2017.0  127.181818          2016.0
...                       ...       ...         ...             ...
Gardner, Ryan   2021        7    2021.0   29.000000          2020.0
Spicer, Phoenix 2022        5    2022.0   27.000000          2021.0
Blanck, James   2023       13    2023.0   25.307692          2022.0
Cadman, Aaron   2023        7    2023.0   24.142857          2022.0
Brown, Nathan   2017       22    2017.0   23.136364          2016.0

[5824 rows x 4 columns]

In [467]:
factors.query('player_name == "Crouch, Matt"')

,player_name,matches_x,year_played,years_active,year,position_grouped,position_grouped_simple,player_height_cm_1yr,player_weight_kg_1yr,age_1yr,...,score_launches_5yr,shots_at_goal_5yr,spoils_5yr,afl_fantasy_score_5yr,matches_y,year_int,avg_score,predicted_year,fantasy_score_yearly_change,fantasy_score_inyear_change
1002,"Crouch, Matt",22,2016.0,8,2016,C,Centres,183.0,85.0,21.0,...,1.472222,0.675676,0.361111,88.972973,25.0,2017.0,108.360000,2016.0,5.572482,25.916667
1003,"Crouch, Matt",25,2017.0,8,2017,C,Centres,183.0,85.0,22.0,...,1.622951,0.709677,0.426230,96.790323,17.0,2018.0,109.117647,2017.0,10.512542,6.100000
1004,"Crouch, Matt",17,2018.0,8,2018,C,Centres,183.0,85.0,23.0,...,1.602564,0.645570,0.371795,99.443038,18.0,2019.0,109.111111,2018.0,5.305147,7.250000
1005,"Crouch, Matt",18,2019.0,8,2019,C,Centres,183.0,85.0,24.0,...,1.591398,0.638298,0.322581,102.042553,NaN,NaN,NaN,NaN,0.311111,-1.688312
1006,"Crouch, Matt",11,2022.0,8,2022,C,Centres,183.0,85.0,27.0,...,1.478261,0.434783,0.173913,104.043478,6.0,2023.0,97.500000,2022.0,0.000000,6.821429
1007,"Crouch, Matt",6,2023.0,8,2023,C,Centres,183.0,85.0,28.0,...,1.428571,0.457143,0.200000,100.457143,NaN,NaN,NaN,NaN,6.205882,0.000000


In [468]:
## Join on response variable - rank by most recent AFL score available (e.g if a player didn't play in 2015, use their 2014 factors to predict the score in 2016)

# factors_final = factors.merge(response_var, left_on=['player_name'], right_on=['player_name'], how='left')\
#         .query(' not predicted_year < year_played')\
        
        
# factors_final['season_rank'] = factors_final.groupby(['player_name','year_played'])['predicted_year'].rank(method="first", ascending=False)

# ## Test that for Elliot Yeo, his 2021 factors predict his 2023 score
# factors_final.query('player_name == "Yeo, Elliot"').query('season_rank == 1')
# # left_on=['player_name', 'year_played'], right_on=['player_name', 'predicted_year'], how='left')

## Create factor that scales up young players based on expected YoY growth

In [469]:
## Calculate average season growth based on player age

player_growth = factors.dropna(axis= 0, how='any').groupby(['age_1yr']).agg({'fantasy_score_yearly_change': 'mean'})
player_growth_pos = factors.dropna(axis= 0, how='any').groupby(['age_1yr', 'position_grouped_simple']).agg({'fantasy_score_yearly_change': 'mean'})

player_growth_pos.rename(columns={'fantasy_score_yearly_change':'fantasy_score_yearly_change_exp'}, inplace=True)

# Add to factors
factors = factors.merge(player_growth_pos, on=['age_1yr', 'position_grouped_simple'], how='left')\

# Fill nulls with 0
factors['fantasy_score_yearly_change_exp'] = (factors['fantasy_score_yearly_change_exp']).fillna(0)

# Fill blank factors AFL average
# factors['age_1yr'] = (factors['age_1yr']).fillna(26)
factors['player_weight_kg_1yr'] = (factors['player_weight_kg_1yr']).fillna(88)
factors['player_height_cm_1yr'] = (factors['player_height_cm_1yr']).fillna(189)

player_growth_pos

fantasy_score_yearly_change_exp
age_1yr position_grouped_simple                                 
18.0    Wings_Flank                                     0.000000
19.0    Centres                                         0.000000
        Key_Position                                   -0.013793
        Ruck                                            0.000000
        Wings_Flank                                     0.047794
...                                                          ...
34.0    Ruck                                           -7.045455
        Wings_Flank                                    -3.000000
35.0    Centres                                        -0.453265
        Key_Position                                   -1.828011
36.0    Centres                                         0.055303

[68 rows x 1 columns]

# Export factors

In [470]:
print(factors.shape)
print(factors.head(10))
factors.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\factors.csv',mode='w+')

(5746, 170)
    player_name  matches_x  year_played  years_active  year position_grouped  \
0   Aarts, Jake         19       2021.0             2  2021              HFF   
1  Ablett, Gary         20       2012.0             8  2012                C   
2  Ablett, Gary         21       2013.0             8  2013                C   
3  Ablett, Gary         14       2014.0             8  2014                C   
4  Ablett, Gary          5       2015.0             8  2015                C   
5  Ablett, Gary         14       2016.0             8  2016                C   
6  Ablett, Gary         14       2017.0             8  2017                C   
7  Ablett, Gary         19       2018.0             8  2018                C   
8  Ablett, Gary         24       2019.0             8  2019               FP   
9  Acres, Blake         15       2016.0             9  2016                W   

  position_grouped_simple  player_height_cm_1yr  player_weight_kg_1yr  \
0             Wings_Flank         